In [0]:
import pandas as pd
df_train = pd.read_csv("Train_data.csv")

In [0]:
import re, time
import nltk
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer, word_tokenize

def base_word(word):
    wordlemmatizer = WordNetLemmatizer()
    return wordlemmatizer.lemmatize(word) 
def word_extractor(text_input):
    commonwords = stopwords.words('english')
    text = re.sub(r'([a-z])\1+', r'\1\1',text_input) #substitute multiple letter by two
    words = ""
    wordtokens = [ base_word(word.lower()) for word in word_tokenize(text) ]
    
    for word in wordtokens:
        if word not in commonwords: #and len(word)>3:  #delete stopwords and emogis
            words+=" "+word
    if len(words) == 0:
        print(text_input)
    return words

In [0]:
train_lemm = []
test_lemm = []
for tweet in df_train.tweet_content:
    train_lemm.append(word_extractor(tweet))

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer= CountVectorizer(ngram_range=(1, 1), binary=False)
vectorizer.fit(train_lemm)

features_train = vectorizer.transform(train_lemm)

# SMOTE

In [137]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# setting up testing and training sets
X_train, X_test, y_train, y_test = train_test_split(features_train, df_train.harassment, test_size=0.25, random_state=27,shuffle = True)

sm = SMOTE(random_state=27, ratio=1.0)
X_train, y_train = sm.fit_sample(X_train, y_train)


from sklearn.ensemble import RandomForestClassifier

#smote = RandomForestClassifier(n_estimators=10).fit(X_train, y_train)
smote = LogisticRegression(solver='liblinear').fit(X_train, y_train)
smote.fit(X_train, y_train)

smote_pred = smote.predict(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [138]:
smote_pred

array([0, 0, 0, ..., 1, 0, 0])

In [139]:
smote.score(X_test,y_test)

0.8401122019635343

In [140]:
from sklearn.metrics import f1_score
f1_score(y_test,smote_pred)

0.4411764705882353

In [0]:
df_test = pd.read_csv("Test_input.csv")

In [0]:
test_lemm = []
for tweet in df_test.tweet_content:
    test_lemm.append(word_extractor(tweet))

In [0]:
features_test = vectorizer.transform(test_lemm)

In [0]:
smote_pred_test = smote.predict(features_test)

In [145]:
smote_pred_test

array([1, 0, 1, ..., 1, 1, 0])

In [0]:
submission = open('submission.csv','w')
submission.write("id,harassment\n")
for i in range(len(df_test.id)):
    submission.write(str(df_test.id[i])+","+str(smote_pred_test[i])+"\n")

#OVERSAMPLING

In [120]:
from sklearn.utils import resample

# Separate input features and target
y = df_train.harassment
X = df_train.tweet_content

# setting up testing and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27, shuffle = True)

# concatenate our training data back together
X = pd.concat([X_train, y_train], axis=1)

# separate minority and majority classes
not_H = X[X.harassment==0]
H = X[X.harassment==1]

# upsample minority
H_upsampled = resample(H,
                          replace=True, # sample with replacement
                          n_samples=len(not_H), # match number in majority class
                          random_state=27) # reproducible results

# combine majority and upsampled minority
upsampled = pd.concat([not_H, H_upsampled])

# check new class counts
upsampled.harassment.value_counts()

1    3874
0    3874
Name: harassment, dtype: int64

In [0]:
y_train = upsampled.harassment
X_train = upsampled.drop('harassment', axis=1)

train_lemm = []
for tweet in X_train.tweet_content:
    train_lemm.append(word_extractor(tweet))

from sklearn.feature_extraction.text import CountVectorizer

vectorizer= CountVectorizer(ngram_range=(1, 1), binary=False)
vectorizer.fit(train_lemm)

features_train = vectorizer.transform(train_lemm)


upsampled = LogisticRegression(solver='liblinear').fit(features_train, y_train)

#upsampled_pred = upsampled.predict(X_test)

In [0]:
df_test = pd.read_csv("Test_input.csv")

In [0]:
test_lemm = []
for tweet in df_test.tweet_content:
    test_lemm.append(word_extractor(tweet))

In [0]:
features_test = vectorizer.transform(test_lemm)

In [0]:
smote_pred_test = upsampled.predict(features_test)

In [0]:
smote_pred_test

In [0]:
submission = open('submission.csv','w')
submission.write("id,harassment\n")
for i in range(len(df_test.id)):
    submission.write(str(df_test.id[i])+","+str(smote_pred_test[i])+"\n")

#UNDERSAMPLING

In [76]:
from sklearn.utils import resample

# Separate input features and target
y = df_train.harassment
X = df_train.tweet_content

# setting up testing and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27, shuffle = True)

# concatenate our training data back together
X = pd.concat([X_train, y_train], axis=1)

# separate minority and majority classes
not_H = X[X.harassment==0]
H = X[X.harassment==1]

# upsample minority
not_H_downsampled = resample(not_H,
                                replace = False, # sample without replacement
                                n_samples = len(H), # match minority n
                                random_state = 27) # reproducible results

# combine minority and downsampled majority
downsampled = pd.concat([not_H_downsampled, H])

# check new class counts
downsampled.harassment.value_counts()

1    403
0    403
Name: harassment, dtype: int64

In [0]:
y_train = downsampled.harassment
X_train = downsampled.drop('harassment', axis=1)

train_lemm = []
for tweet in X_train.tweet_content:
    train_lemm.append(word_extractor(tweet))

from sklearn.feature_extraction.text import CountVectorizer

vectorizer= CountVectorizer(ngram_range=(1, 1), binary=False)
vectorizer.fit(train_lemm)

features_train = vectorizer.transform(train_lemm)


upsampled = LogisticRegression(solver='liblinear').fit(features_train, y_train)

#upsampled_pred = upsampled.predict(X_test)

In [0]:
df_test = pd.read_csv("Test_input.csv")

In [0]:
test_lemm = []
for tweet in df_test.tweet_content:
    test_lemm.append(word_extractor(tweet))

In [0]:
features_test = vectorizer.transform(test_lemm)

In [0]:
smote_pred_test = upsampled.predict(features_test)

In [0]:
smote_pred_test

In [0]:
submission = open('submission.csv','w')
submission.write("id,harassment\n")
for i in range(len(df_test.id)):
    submission.write(str(df_test.id[i])+","+str(smote_pred_test[i])+"\n")